In [ ]:
import torch
from torchvision import transforms
import numpy as np
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import glob

from src.hog.hog import test_hog, load_hog
from src.hog.hog_op import compute_hog_character

from src.utils.load_data import get_test_data
from src.utils.char_to_int import convert_int_to_char, convert_char_to_int

from src.nn.model import CaptchaClassifierBigger

# Load Data

In [ ]:
X_test, y_test = get_test_data()

test_images_ground_truth_files = sorted(glob.glob('./dados/teste/*.jpg'))
test_images_ground_truth = [cv2.imread(img, cv2.IMREAD_GRAYSCALE) for img in test_images_ground_truth_files ]

test_label_files = sorted(glob.glob('./dados/labels10k/*.txt'))[9000:]
test_labels = [open(label_file).read().strip() for label_file in test_label_files]

# Hog

In [ ]:
hog_classifier_path = 'models/captcha_classifier_hog.pkl'
classifier = load_hog(hog_classifier_path)

accuracy = test_hog(classifier, X_test, y_test)

In [ ]:
def results_hog(idx_image):
    offset = 6 * idx_image
    
    preds = []
    for i in range(6):
        features = compute_hog_character(X_test[i+offset])
        features = features.reshape(1, -1)
        pred = classifier.predict(features)
        preds.append(pred[0])
            
    pred_string = ''.join(preds)
    print("Predictions:", pred_string)
    print("Ground Truth label: ", test_labels[offset//6])
    
    num_errors = 0
    for i in range(len(pred_string)):
        if pred_string[i] != test_labels[offset//6][i]:
            num_errors += 1

    if not num_errors:
        print('Acertou todos')
    else:
        print(f'Errou {num_errors} vezes')

In [ ]:
results_hog(31)

# NN

In [ ]:
model = CaptchaClassifierBigger()
model.load_state_dict(torch.load('models/nn_model_bigger.pth'))
model.eval()
model.to('cuda')

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

def results_nn(idx_image):
    offset = 6 * idx_image
    
    fig = plt.figure(figsize=(10, 6))
    
    ax1 = plt.subplot2grid((2, 1), (0, 0), rowspan=1)
    ax1.imshow(test_images_ground_truth[offset//6], cmap='gray')
    ax1.axis('off')
    
    preds = []
    for i in range(6):
        ax = plt.subplot2grid((2, 6), (1, i))
        ax.imshow(X_test[i+offset], cmap='gray')
        ax.axis('off')
        
        with torch.no_grad():
            tensor = transform(X_test[i+offset]).to('cuda')
            outputs = model(tensor.unsqueeze(0))
            _, pred = torch.max(outputs, 1)
            char_pred = convert_int_to_char(int(pred))
            preds.append(char_pred)

    plt.tight_layout()
    
    plt.savefig('./images/ground_truth_vs_filtered.png', bbox_inches='tight')
    
    pred_string = ''.join(preds)
    print("Predictions:", pred_string)
    print("Ground Truth label: ", test_labels[offset//6])
    
    num_errors = 0
    for i in range(len(pred_string)):
        if pred_string[i] != test_labels[offset//6][i]:
            num_errors += 1

    if not num_errors:
        print('Acertou todos')
    else:
        print(f'Errou {num_errors} vezes')


In [ ]:
results_nn(315)

# Plot accuracy by number of digits

In [ ]:
def predict_hog(data):
    features = compute_hog_character(data)
    features = features.reshape(1, -1)
    pred = classifier.predict(features)
    return pred[0]

def predict_nn(data):
    tensor = transform(data).to('cuda')
    outputs = model(tensor.unsqueeze(0))
    _, pred = torch.max(outputs, 1)
    char_pred = convert_int_to_char(int(pred))
    return char_pred

In [ ]:
def accuracy_num_digits(predict_function):
    
    accuracies = []
    recognitions = []
    
    for num_digits in [1, 2, 3, 4, 5, 6]:
        digit_acc = []
        digit_recog = []
        for idx in range(len(test_labels)):
            offset = 6 * idx
        
            preds = []
            num_errors = 0
            recog = 1
            
            indices = list(range(num_digits))
            np.random.shuffle(indices)
            
            for i in indices:
                with torch.no_grad():
                    pred = predict_function(X_test[i+offset])
                    preds.append(pred)
                    
                    if pred != test_labels[offset//6][i]:
                        num_errors += 1
            
            # if idx % 250 == 0:
            #     ordered_label = ''.join(np.array(list(test_labels[offset//6]))[indices])
            #     print(''.join(preds), ordered_label, num_errors, num_digits, test_labels[offset//6])
            
            digit_acc.append((1 - num_errors/num_digits) * 100)
            
            if num_errors > 0:
                recog = 0
                
            digit_recog.append(recog)
                    
        accuracies.append(np.mean(digit_acc))
        recognitions.append(np.sum(digit_recog)/len(test_labels))
        
    return accuracies, recognitions
            

In [ ]:
accuracies_hog, recognitions_hog = accuracy_num_digits(predict_hog)

In [ ]:
accuracies_nn, recognitions_nn = accuracy_num_digits(predict_nn)

In [ ]:
def plot_by_num_digits(accuracies_hog, accuracies_nn, feature):
    x = [1, 2, 3, 4, 5, 6]
    
    plt.figure(figsize=(10, 6))

    sns.lineplot(x=x, y=accuracies_hog, marker='o', label='HOG', color='blue')

    sns.lineplot(x=x, y=accuracies_nn, marker='o', label='NN', color='orange')
    
    plt.grid(True)
    
    plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y:.2f}%'))

    # Add title and labels
    plt.title(f'{feature} Comparison of HOG and NN Models by Number of Digits')
    plt.xlabel('Number of Digits')
    plt.ylabel(f'{feature} (%)')

    plt.legend(title='Model')
    
    plt.savefig(f'./images/{feature}_hog_vs_nn.png', bbox_inches='tight')

    plt.show()

In [ ]:
plot_by_num_digits(accuracies_hog, accuracies_nn, 'Accuracy')

In [ ]:
recognitions_hog = recognition_hog

In [ ]:
plot_by_num_digits(recognitions_hog, recognitions_nn, 'Recognition')